In [112]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import netCDF4 as nc
import xarray as xr
import wrf

## Functions

In [113]:
def format_times(da):
    ts = pd.to_datetime(da.time.values)
    tsPDT = ts - pd.DateOffset(hours=7)
    da['time'] = tsPDT
    return da

def extract_rh(wrf_ds, lat, long):
    lats = wrf_ds['XLAT'].values[:, 0]
    lons = wrf_ds['XLONG'].values[0, :]
    times = wrf_ds.Time.values
    rh2 = wrf_ds.values

    rh_da = xr.DataArray(
        data = rh2,
        dims=['time', 'x', 'y'],
        coords = {'time' : times, 'y' : lons, 'x' : lats}
    )

    rh_da = rh_da.sel(x=[lat], y=[long], method='nearest').squeeze()
    rh_da = format_times(rh_da)

    times = pd.to_datetime(rh_da['time'].values)
    rh_df = pd.DataFrame(data={'time (PST)' : times, 'wrf rh' : rh_da.values})
    rh_df.set_index('time (PST)')

    return rh_df

In [114]:
## Prepare data

In [115]:
# Load in data
root_dir  = Path().resolve().parent
data_dir = root_dir / 'data'
sauc_fpath = data_dir / 'observational' / 'aggregated-datasets' / 'sauc-hourly.csv'
wrf_fpath = data_dir / 'geospatial' / 'wrf-vertical-profile' / 'wrfout-2008-08-vertprofile.nc'

sauc_df = pd.read_csv(sauc_fpath)
wrf_nc = nc.Dataset(wrf_fpath)
wrf_rh_ds = wrf.getvar(wrf_nc, 'rh2', timeidx=wrf.ALL_TIMES)


In [116]:
# Extract RH2 at Sauces
sauclat = 34.001033
sauclong = -119.817817
wrf_df = extract_rh(wrf_rh_ds, sauclat, sauclong)
wrf_df.head(3)


,time (PST),wrf rh
0,2008-08-16 17:00:00,70.502312
1,2008-08-16 18:00:00,77.249458
2,2008-08-16 19:00:00,86.178139


In [117]:
# Subset observational
sauc_subset = sauc_df[['time (PST)', 'fog tips', 'relative humidity (%)', 'rain (mm)']]
sauc_subset = sauc_subset.rename({'relative humidity (%)' : 'observed rh'}, axis=1)
sauc_subset.set_index('time (PST)', inplace=True)
sauc_subset

,fog tips,observed rh,rain (mm)
time (PST),,,
2003-12-19 16:00:00,0.0,27.5400,0.0
2003-12-19 17:00:00,0.0,27.0500,0.0
2003-12-19 18:00:00,0.0,32.8600,0.0
2003-12-19 19:00:00,0.0,60.6800,0.0
2003-12-19 20:00:00,0.0,49.7900,0.0
...,...,...,...
2010-02-18 11:00:00,0.0,11.6475,0.0
2010-02-18 12:00:00,0.0,11.0000,0.0
2010-02-18 13:00:00,0.0,9.1825,0.0


In [128]:
wrf_df.index = wrf_df['time (PST)'].values

In [130]:
pd.concat([sauc_subset, wrf_df], join='inner')

""
2003-12-19 16:00:00
2003-12-19 17:00:00
2003-12-19 18:00:00
2003-12-19 19:00:00
2003-12-19 20:00:00
...
2008-08-25 12:00:00
2008-08-25 13:00:00
2008-08-25 14:00:00
2008-08-25 15:00:00
